In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

pd.set_option('display.max_columns', None)

df=pd.read_csv('game_data.csv')

In [41]:

# Quarterbacks who are playing this 2023 season
current_quarterbacks = ['L.Jackson', 'J.Allen', 'J.Browning', 'D.Thompson-Robinson', 'R.Wilson', 'C.J. Stroud', 'G.Minshew', 'T.Lawrence', 'P.Mahomes', 'A.Oconnell', 'J.Herbert', 'T.Tagovailoa', 'M.Jones', 'Tim Boyle', 'K.Pickett', 'W.Levis', 'D.Ridder', 'K.Murray', 'B.Young', 'J.Fields', 'D.Prescott', 'J.Goff', 'Jordan Love', 'M.Stafford', 'J.Dobbs', 'J.Winston', 'T.DeVito', 'J.Hurts', 'B.Purdy', 'G.Smith', 'B.Mayfield', 'S.Howell']

# Filter the DataFrame
df = df[df["passer_player_name"].isin(current_quarterbacks)]


df.head(5)

,passer_player_name,week,season,posteam,defteam,complete_pass,incomplete_pass,interception,qb_hit,sack,pass_touchdown,passing_yards,pass_attempts
580,B.Mayfield,1,2019,CLE,TEN,25.0,10.0,3.0,6.0,5.0,1.0,285.0,35.0
581,B.Mayfield,1,2020,CLE,BAL,21.0,17.0,1.0,6.0,2.0,1.0,189.0,38.0
582,B.Mayfield,1,2021,CLE,KC,21.0,6.0,1.0,5.0,2.0,0.0,321.0,27.0
583,B.Mayfield,1,2022,CAR,CLE,16.0,10.0,1.0,4.0,4.0,1.0,235.0,26.0
584,B.Mayfield,1,2023,TB,MIN,21.0,13.0,0.0,2.0,1.0,2.0,173.0,34.0


Feature Engineering

In [2]:
df['pass_attempts']=df['complete_pass']+df["incomplete_pass"]


In [3]:
defense_df = df[df['season'] == 2023].groupby('defteam').agg({
    'complete_pass': 'mean',
    'incomplete_pass': 'mean',
    'interception': 'mean',
    'qb_hit': 'mean',
    'sack': 'mean',
    'pass_touchdown': 'mean',
    'passing_yards': 'mean'
}).reset_index()

## Creates a new dataframe of the defending teams average allowed stats for 2023


In [4]:
new_names = {
    'complete_pass': 'def_complete_pass_allowed',
    'incomplete_pass': 'def_avg_incomplete_pass',
    'interception': 'def_avg_ints',
    'qb_hit': 'def_avg_qb_hit',
    'sack': 'def_avg_sack',
    'pass_touchdown': 'def_avg_td_allowed',
    'passing_yards': 'def_avg_yards_allowed'
}

# Renaming them for easier understanding when we merge this with the game DF
defense_df = defense_df.rename(columns=new_names)

In [5]:
defense_df=defense_df.drop(columns=['def_avg_incomplete_pass'])  ## This is redundant because it's just the inverse of passrate
defense_df.head(5)

,defteam,def_complete_pass_allowed,def_avg_ints,def_avg_qb_hit,def_avg_sack,def_avg_td_allowed,def_avg_yards_allowed
0,ARI,17.400000,0.600000,3.066667,2.133333,1.400000,190.266667
1,ATL,17.461538,0.461538,4.769231,1.692308,1.307692,188.384615
2,BAL,18.000000,0.733333,5.600000,3.133333,0.666667,162.266667
3,BUF,17.400000,0.733333,5.333333,2.733333,1.066667,178.266667
4,CAR,15.071429,0.357143,3.428571,1.285714,0.928571,150.142857


In [6]:
fulldf =  pd.merge(df, defense_df, on='defteam')


In [7]:
tempdf=fulldf[fulldf['season'] == 2023]  

In [8]:
## This will calculate average stats for the quarterbacks, on games played this season so far

tempdf = tempdf.groupby('passer_player_name').agg({
    'complete_pass': 'mean',
    'passing_yards': 'mean',
    'pass_attempts': 'mean',
    'interception': 'mean'
}).reset_index().rename(columns={'complete_pass': '2023_qb_completion_avg', 'passing_yards': '2023_qb_yards_avg', 'pass_attempts': '2023_qb_attempts_avg', 'interception': '2023_qb_int_avg'})

In [9]:
fulldf =  pd.merge(fulldf, tempdf, on='passer_player_name')
fulldf.head(4)

,passer_player_name,week,season,posteam,defteam,complete_pass,incomplete_pass,interception,qb_hit,sack,pass_touchdown,passing_yards,pass_attempts,def_complete_pass_allowed,def_avg_ints,def_avg_qb_hit,def_avg_sack,def_avg_td_allowed,def_avg_yards_allowed,2023_qb_completion_avg,2023_qb_yards_avg,2023_qb_attempts_avg,2023_qb_int_avg
0,A.Dalton,1,2016,CIN,NYJ,23.0,6.0,1.0,9.0,7.0,1.0,366.0,29.0,18.333333,1.000000,5.583333,2.333333,1.083333,182.083333,34.0,361.0,58.0,0.0
1,A.Dalton,13,2019,CIN,NYJ,22.0,15.0,0.0,2.0,1.0,1.0,243.0,37.0,18.333333,1.000000,5.583333,2.333333,1.083333,182.083333,34.0,361.0,58.0,0.0
2,A.Dalton,12,2021,CHI,DET,24.0,14.0,1.0,4.0,1.0,1.0,317.0,38.0,18.230769,0.615385,4.538462,1.769231,1.461538,207.461538,34.0,361.0,58.0,0.0
3,A.Dalton,16,2017,CIN,DET,27.0,13.0,1.0,3.0,3.0,1.0,238.0,40.0,18.230769,0.615385,4.538462,1.769231,1.461538,207.461538,34.0,361.0,58.0,0.0


In [10]:
fulldf = fulldf[fulldf['pass_attempts'] >= 15]

## Less than 15 pass attempts means they didnt play the whole game, probably because of injury, a big score difference, or it was a trick play and they only threw once



In [60]:
fulldf.to_csv('current_qb.csv', index = False)